In [ ]:
import numpy as np
import pandas as pd
import os
import gzip
import io
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 2. Классификация сообщений сервера



In [ ]:
# vibo: читаем предобработанный лог (index_col=0 считываем индексы тоже)
data_logs = pd.read_csv('/content/drive/MyDrive/minecraft/preproc_logs.csv', index_col=0)

Посмотрим на заголовки сообщений сервера, с которым будем работать. Выделим из него типы сообщений, которые отвечают за степень важности сообщения (`INFO`, `WARN`, `ERROR`) и категории сообщений:
* `Async Chat Thread` - чат;
* `Server thread` - игровой процесс;
* `ServerMain` - системные сообщения;
* `User Authenticator` - индентификация пользователя;
* `Worker-Main` - процесс построения мира;
* `WorldGuard Region I/O` - плагин WorldGuard.

In [ ]:
data_logs["serv_teg"].value_counts()

Server thread/INFO              5294
Worker-Main-3/INFO               157
Worker-Main-4/INFO               156
Server thread/WARN               150
Worker-Main-1/INFO               131
Worker-Main-6/INFO               120
Worker-Main-5/INFO               116
Worker-Main-7/INFO               107
Worker-Main-2/INFO               104
ServerMain/INFO                   84
User Authenticator #1/INFO        22
User Authenticator #2/INFO        13
WorldGuard Region I/O/INFO        11
User Authenticator #3/INFO        10
User Authenticator #4/INFO         9
User Authenticator #5/INFO         9
User Authenticator #7/INFO         8
User Authenticator #6/INFO         8
Async Chat Thread - #0/INFO        5
Async Chat Thread - #2/INFO        5
User Authenticator #8/INFO         5
Async Chat Thread - #6/INFO        3
Async Chat Thread - #10/INFO       3
User Authenticator #9/INFO         3
User Authenticator #10/INFO        2
Async Chat Thread - #4/INFO        1
ServerMain/ERROR                   1
A

In [ ]:
# vibo: отделяем теги типа INFO, ERROR и WARN
data_logs["serv_teg_info"] = data_logs["serv_teg"].dropna().map(lambda x: x.split('/')[0])
data_logs["serv_teg_TEG"] = data_logs["serv_teg"].dropna().map(lambda x: x.split('/')[1])
data_logs[['date', 'time', 'serv_teg', 'serv_teg_info', 'serv_teg_TEG']]

,date,time,serv_teg,serv_teg_info,serv_teg_TEG
0,2023-02-13,21:31:47,ServerMain/INFO,ServerMain,INFO
1,2023-02-13,21:31:48,ServerMain/ERROR,ServerMain,ERROR
2,2023-02-13,21:31:48,ServerMain/WARN,ServerMain,WARN
3,2023-02-13,21:31:48,ServerMain/INFO,ServerMain,INFO
5,2023-02-13,21:34:03,ServerMain/INFO,ServerMain,INFO
...,...,...,...,...,...
6577,2023-03-09,18:50:50,User Authenticator #5/INFO,User Authenticator #5,INFO
6578,2023-03-09,18:50:50,Server thread/INFO,Server thread,INFO
6579,2023-03-09,18:50:50,Server thread/INFO,Server thread,INFO
6580,2023-03-09,18:52:31,Server thread/INFO,Server thread,INFO


In [ ]:
data_logs["serv_teg_info"].value_counts()

Server thread              5444
Worker-Main-3               157
Worker-Main-4               156
Worker-Main-1               131
Worker-Main-6               120
Worker-Main-5               116
Worker-Main-7               107
Worker-Main-2               104
ServerMain                   86
User Authenticator #1        22
User Authenticator #2        13
WorldGuard Region I          11
User Authenticator #3        10
User Authenticator #4         9
User Authenticator #5         9
User Authenticator #6         8
User Authenticator #7         8
User Authenticator #8         5
Async Chat Thread - #0        5
Async Chat Thread - #2        5
Async Chat Thread - #6        3
Async Chat Thread - #10       3
User Authenticator #9         3
User Authenticator #10        2
Async Chat Thread - #8        1
Async Chat Thread - #4        1
User Authenticator #11        1
User Authenticator #12        1
User Authenticator #13        1
User Authenticator #14        1
Async Chat Thread - #1        1
Name: se

In [ ]:
# vibo: выборка по User Authenticator #1
data_logs[data_logs["serv_teg_info"] == 'User Authenticator #1'].groupby(['date', 'serv_mess'])['serv_mess'].count()

date         serv_mess                                                           
2023-02-13    UUID of player vibO is 426ba0c3-6437-3e82-b36b-ca953fb41a42            2
2023-02-14    UUID of player vibO is 426ba0c3-6437-3e82-b36b-ca953fb41a42            1
2023-02-17    UUID of player YOUTUBE_plaY is 56eb670a-4ed0-3009-9bc9-00d047fbaeff    1
2023-03-03    UUID of player LOLOTUK is 351abe30-df95-37d4-940e-b868831fc026         1
              UUID of player YOUTUBE_plaY is 56eb670a-4ed0-3009-9bc9-00d047fbaeff    1
              UUID of player rex is 74d83d51-5bf6-37fb-8395-9a0590a59259             1
2023-03-04    UUID of player rex is 74d83d51-5bf6-37fb-8395-9a0590a59259             3
2023-03-05    UUID of player YOUTUBE_plaY is 56eb670a-4ed0-3009-9bc9-00d047fbaeff    1
              UUID of player amogoos is bb8d3a15-62ea-3dd7-8df8-776e0930f557         1
              UUID of player vibo is f4a7e80b-36fa-3e7a-814b-9824a8ccd5f9            1
2023-03-06    UUID of player amogoos is bb8d3a15

In [ ]:
# vibo: выборка по User Authenticator #2
data_logs[data_logs["serv_teg_info"] == 'User Authenticator #2'].groupby(['date', 'serv_mess'])['serv_mess'].count()

date         serv_mess                                                               
2023-02-17    UUID of player YOUTUBE_plaY is 56eb670a-4ed0-3009-9bc9-00d047fbaeff        1
2023-03-03    UUID of player LOLOTUK is 351abe30-df95-37d4-940e-b868831fc026             1
              UUID of player YOUTUBE_plaY is 56eb670a-4ed0-3009-9bc9-00d047fbaeff        1
2023-03-05    UUID of player amogoos is bb8d3a15-62ea-3dd7-8df8-776e0930f557             1
              UUID of player amogoos_red15050 is 90c58b0a-047b-3f41-a2eb-0b295facaf2a    1
2023-03-06    UUID of player amogoos is bb8d3a15-62ea-3dd7-8df8-776e0930f557             1
              UUID of player testing1 is 4e9071c3-79ab-3a3a-ab83-2e04eacd6854            1
              UUID of player testing2 is 5e20431e-cf2a-314e-a557-c5525d567366            1
2023-03-07    UUID of player Fredi is 1ee24d02-4446-35e1-84c8-4b785c1bae33               1
2023-03-08    UUID of player Umni4ok100200 is b78a67d1-e667-3841-a463-64fdf66f2484       1
    

In [ ]:
# vibo: плагин WorldGuard выдает свой тег на верхний уровень
data_logs[data_logs["serv_teg_info"] == 'WorldGuard Region I']

,date,time,serv_teg,serv_mess,plug_name,plug_mess,serv_teg_info,serv_teg_TEG
4277,2023-03-06,21:06:20,WorldGuard Region I/O/INFO,NaN,WorldGuard,Region data changes made in 'world' have been ...,WorldGuard Region I,O
4283,2023-03-06,21:08:50,WorldGuard Region I/O/INFO,NaN,WorldGuard,Region data changes made in 'world' have been ...,WorldGuard Region I,O
5018,2023-03-07,18:10:12,WorldGuard Region I/O/INFO,NaN,WorldGuard,Region data changes made in 'world' have been ...,WorldGuard Region I,O
5021,2023-03-07,18:10:42,WorldGuard Region I/O/INFO,NaN,WorldGuard,Region data changes made in 'world' have been ...,WorldGuard Region I,O
5023,2023-03-07,18:11:12,WorldGuard Region I/O/INFO,NaN,WorldGuard,Region data changes made in 'world' have been ...,WorldGuard Region I,O
5039,2023-03-07,18:29:42,WorldGuard Region I/O/INFO,NaN,WorldGuard,Region data changes made in 'world' have been ...,WorldGuard Region I,O
5042,2023-03-07,18:30:42,WorldGuard Region I/O/INFO,NaN,WorldGuard,Region data changes made in 'world' have been ...,WorldGuard Region I,O
5061,2023-03-07,18:42:12,WorldGuard Region I/O/INFO,NaN,WorldGuard,Region data changes made in 'world' have been ...,WorldGuard Region I,O
5066,2023-03-07,18:44:12,WorldGuard Region I/O/INFO,NaN,WorldGuard,Region data changes made in 'world' have been ...,WorldGuard Region I,O
5068,2023-03-07,18:44:42,WorldGuard Region I/O/INFO,NaN,WorldGuard,Region data changes made in 'world' have been ...,WorldGuard Region I,O


In [ ]:
data_logs["serv_teg_TEG"].value_counts()

INFO     6381
WARN      151
O          11
ERROR       1
Name: serv_teg_TEG, dtype: int64

По `/` не разрезать из-за `WorldGuard Region I/O/INFO`, который мешает.

In [ ]:
# vibo: удаляем столбцы
data_logs = data_logs.drop(['serv_teg_info', 'serv_teg_TEG'], axis=1)
data_logs

,date,time,serv_teg,serv_mess,plug_name,plug_mess
0,2023-02-13,21:31:47,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN
1,2023-02-13,21:31:48,ServerMain/ERROR,Failed to load properties from file: server.p...,NaN,NaN
2,2023-02-13,21:31:48,ServerMain/WARN,Failed to load eula.txt,NaN,NaN
3,2023-02-13,21:31:48,ServerMain/INFO,You need to agree to the EULA in order to run...,NaN,NaN
5,2023-02-13,21:34:03,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN
...,...,...,...,...,...,...
6577,2023-03-09,18:50:50,User Authenticator #5/INFO,UUID of player ARTEXX is ee86a614-d39f-3a50-a...,NaN,NaN
6578,2023-03-09,18:50:50,Server thread/INFO,ARTEXX joined the game,NaN,NaN
6579,2023-03-09,18:50:50,Server thread/INFO,ARTEXX[/[0:0:0:0:0:0:0:1,NaN,NaN
6580,2023-03-09,18:52:31,Server thread/INFO,ARTEXX issued server command: /farwand,NaN,NaN


## 2.1 Типы сообщений (INFO, WARN и ERROR)

Выше было видно, что тега всего три - `INFO`, `WARN` и `ERROR`, выделим их в отдельный столбец.

In [ ]:
# vibo: проставили True, там где есть INFO
data_logs['serv_teg_TEG_INFO'] = data_logs['serv_teg'].dropna().map(lambda x: 'INFO' in x)
data_logs[['date', 'time', 'serv_teg', 'serv_teg_TEG_INFO']]

,date,time,serv_teg,serv_teg_TEG_INFO
0,2023-02-13,21:31:47,ServerMain/INFO,True
1,2023-02-13,21:31:48,ServerMain/ERROR,False
2,2023-02-13,21:31:48,ServerMain/WARN,False
3,2023-02-13,21:31:48,ServerMain/INFO,True
5,2023-02-13,21:34:03,ServerMain/INFO,True
...,...,...,...,...
6577,2023-03-09,18:50:50,User Authenticator #5/INFO,True
6578,2023-03-09,18:50:50,Server thread/INFO,True
6579,2023-03-09,18:50:50,Server thread/INFO,True
6580,2023-03-09,18:52:31,Server thread/INFO,True


In [ ]:
# vibo: нишли индексы строк с INFO
ind_INFO = data_logs[data_logs['serv_teg_TEG_INFO'] == True].index
ind_INFO

Int64Index([   0,    3,    5,    6,    7,    8,    9,   10,   11,   12,
            ...
            6572, 6573, 6574, 6575, 6576, 6577, 6578, 6579, 6580, 6581],
           dtype='int64', length=6392)

In [ ]:
# vibo: заменили True на INFO
data_logs['serv_teg_TEG_INFO'] = data_logs[data_logs.index.isin(ind_INFO)]['serv_teg_TEG_INFO'].map(lambda x: 'INFO')
data_logs[['date', 'time', 'serv_teg', 'serv_teg_TEG_INFO']]

,date,time,serv_teg,serv_teg_TEG_INFO
0,2023-02-13,21:31:47,ServerMain/INFO,INFO
1,2023-02-13,21:31:48,ServerMain/ERROR,NaN
2,2023-02-13,21:31:48,ServerMain/WARN,NaN
3,2023-02-13,21:31:48,ServerMain/INFO,INFO
5,2023-02-13,21:34:03,ServerMain/INFO,INFO
...,...,...,...,...
6577,2023-03-09,18:50:50,User Authenticator #5/INFO,INFO
6578,2023-03-09,18:50:50,Server thread/INFO,INFO
6579,2023-03-09,18:50:50,Server thread/INFO,INFO
6580,2023-03-09,18:52:31,Server thread/INFO,INFO


In [ ]:
# vibo: делаем тоже самое для WARN и ERROR
data_logs['serv_teg_TEG_WARN'] = data_logs[data_logs['serv_teg'] != 'INFO']['serv_teg'].map(lambda x: 'WARN' in x)
ind_WARN = data_logs[data_logs['serv_teg_TEG_WARN'] == True].index
data_logs['serv_teg_TEG_WARN'] = data_logs[data_logs.index.isin(ind_WARN)]['serv_teg_TEG_WARN'].map(lambda x: 'WARN')

data_logs['serv_teg_TEG_ERROR'] = data_logs['serv_teg'].map(lambda x: 'ERROR' in x)
ind_ERROR = data_logs[data_logs['serv_teg_TEG_ERROR'] == True].index
data_logs['serv_teg_TEG_ERROR'] = data_logs[data_logs.index.isin(ind_ERROR)]['serv_teg_TEG_ERROR'].map(lambda x: 'ERROR')

data_logs

,date,time,serv_teg,serv_mess,plug_name,plug_mess,serv_teg_TEG_INFO,serv_teg_TEG_WARN,serv_teg_TEG_ERROR
0,2023-02-13,21:31:47,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO,NaN,NaN
1,2023-02-13,21:31:48,ServerMain/ERROR,Failed to load properties from file: server.p...,NaN,NaN,NaN,NaN,ERROR
2,2023-02-13,21:31:48,ServerMain/WARN,Failed to load eula.txt,NaN,NaN,NaN,WARN,NaN
3,2023-02-13,21:31:48,ServerMain/INFO,You need to agree to the EULA in order to run...,NaN,NaN,INFO,NaN,NaN
5,2023-02-13,21:34:03,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6577,2023-03-09,18:50:50,User Authenticator #5/INFO,UUID of player ARTEXX is ee86a614-d39f-3a50-a...,NaN,NaN,INFO,NaN,NaN
6578,2023-03-09,18:50:50,Server thread/INFO,ARTEXX joined the game,NaN,NaN,INFO,NaN,NaN
6579,2023-03-09,18:50:50,Server thread/INFO,ARTEXX[/[0:0:0:0:0:0:0:1,NaN,NaN,INFO,NaN,NaN
6580,2023-03-09,18:52:31,Server thread/INFO,ARTEXX issued server command: /farwand,NaN,NaN,INFO,NaN,NaN


In [ ]:
# vibo: собираем в все в один столбик ffill (все собралось в последний столбец)
data_logs[['serv_teg_TEG_INFO', 'serv_teg_TEG_WARN', 'serv_teg_TEG_ERROR']] = data_logs[['serv_teg_TEG_INFO', 'serv_teg_TEG_WARN', 'serv_teg_TEG_ERROR']].ffill(axis=1)
data_logs

,date,time,serv_teg,serv_mess,plug_name,plug_mess,serv_teg_TEG_INFO,serv_teg_TEG_WARN,serv_teg_TEG_ERROR
0,2023-02-13,21:31:47,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO,INFO,INFO
1,2023-02-13,21:31:48,ServerMain/ERROR,Failed to load properties from file: server.p...,NaN,NaN,NaN,NaN,ERROR
2,2023-02-13,21:31:48,ServerMain/WARN,Failed to load eula.txt,NaN,NaN,NaN,WARN,WARN
3,2023-02-13,21:31:48,ServerMain/INFO,You need to agree to the EULA in order to run...,NaN,NaN,INFO,INFO,INFO
5,2023-02-13,21:34:03,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO,INFO,INFO
...,...,...,...,...,...,...,...,...,...
6577,2023-03-09,18:50:50,User Authenticator #5/INFO,UUID of player ARTEXX is ee86a614-d39f-3a50-a...,NaN,NaN,INFO,INFO,INFO
6578,2023-03-09,18:50:50,Server thread/INFO,ARTEXX joined the game,NaN,NaN,INFO,INFO,INFO
6579,2023-03-09,18:50:50,Server thread/INFO,ARTEXX[/[0:0:0:0:0:0:0:1,NaN,NaN,INFO,INFO,INFO
6580,2023-03-09,18:52:31,Server thread/INFO,ARTEXX issued server command: /farwand,NaN,NaN,INFO,INFO,INFO


In [ ]:
# vibo: переименуем столбец
data_logs = data_logs.rename(columns={"serv_teg_TEG_ERROR": "serv_teg_TEG"})
# vibo: удаляем промежуточные столбцы
data_logs = data_logs.drop(['serv_teg_TEG_INFO', 'serv_teg_TEG_WARN'], axis=1)
data_logs

,date,time,serv_teg,serv_mess,plug_name,plug_mess,serv_teg_TEG
0,2023-02-13,21:31:47,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO
1,2023-02-13,21:31:48,ServerMain/ERROR,Failed to load properties from file: server.p...,NaN,NaN,ERROR
2,2023-02-13,21:31:48,ServerMain/WARN,Failed to load eula.txt,NaN,NaN,WARN
3,2023-02-13,21:31:48,ServerMain/INFO,You need to agree to the EULA in order to run...,NaN,NaN,INFO
5,2023-02-13,21:34:03,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO
...,...,...,...,...,...,...,...
6577,2023-03-09,18:50:50,User Authenticator #5/INFO,UUID of player ARTEXX is ee86a614-d39f-3a50-a...,NaN,NaN,INFO
6578,2023-03-09,18:50:50,Server thread/INFO,ARTEXX joined the game,NaN,NaN,INFO
6579,2023-03-09,18:50:50,Server thread/INFO,ARTEXX[/[0:0:0:0:0:0:0:1,NaN,NaN,INFO
6580,2023-03-09,18:52:31,Server thread/INFO,ARTEXX issued server command: /farwand,NaN,NaN,INFO


In [ ]:
# vibo: проверим сумму строк
serv_teg_TEG = data_logs["serv_teg_TEG"].value_counts().to_frame()
serv_teg_TEG.agg('sum', axis=0)

serv_teg_TEG    6544
dtype: int64

In [ ]:
# vibo: выделили индексы
serv_teg_TEG.index

Index(['INFO', 'WARN', 'ERROR'], dtype='object')

In [ ]:
data_logs

,date,time,serv_teg,serv_mess,plug_name,plug_mess,serv_teg_TEG
0,2023-02-13,21:31:47,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO
1,2023-02-13,21:31:48,ServerMain/ERROR,Failed to load properties from file: server.p...,NaN,NaN,ERROR
2,2023-02-13,21:31:48,ServerMain/WARN,Failed to load eula.txt,NaN,NaN,WARN
3,2023-02-13,21:31:48,ServerMain/INFO,You need to agree to the EULA in order to run...,NaN,NaN,INFO
5,2023-02-13,21:34:03,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO
...,...,...,...,...,...,...,...
6577,2023-03-09,18:50:50,User Authenticator #5/INFO,UUID of player ARTEXX is ee86a614-d39f-3a50-a...,NaN,NaN,INFO
6578,2023-03-09,18:50:50,Server thread/INFO,ARTEXX joined the game,NaN,NaN,INFO
6579,2023-03-09,18:50:50,Server thread/INFO,ARTEXX[/[0:0:0:0:0:0:0:1,NaN,NaN,INFO
6580,2023-03-09,18:52:31,Server thread/INFO,ARTEXX issued server command: /farwand,NaN,NaN,INFO


Теперь нужно убрать теги (`INFO`, `WARN`, `ERROR`) из столбца `serv_teg`

In [ ]:
# vibo: теперь выделим из serv_teg - остальную информацию
# vibo: можно было также выделять теги
data_logs['serv_teg_INF'] = data_logs['serv_teg'].dropna().map(lambda x: x[::-1].split('/')[1:])
data_logs['serv_teg_INF'] = data_logs['serv_teg_INF'].dropna().map(lambda x: ('/'.join(x))[::-1])
data_logs

,date,time,serv_teg,serv_mess,plug_name,plug_mess,serv_teg_TEG,serv_teg_INF
0,2023-02-13,21:31:47,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO,ServerMain
1,2023-02-13,21:31:48,ServerMain/ERROR,Failed to load properties from file: server.p...,NaN,NaN,ERROR,ServerMain
2,2023-02-13,21:31:48,ServerMain/WARN,Failed to load eula.txt,NaN,NaN,WARN,ServerMain
3,2023-02-13,21:31:48,ServerMain/INFO,You need to agree to the EULA in order to run...,NaN,NaN,INFO,ServerMain
5,2023-02-13,21:34:03,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO,ServerMain
...,...,...,...,...,...,...,...,...
6577,2023-03-09,18:50:50,User Authenticator #5/INFO,UUID of player ARTEXX is ee86a614-d39f-3a50-a...,NaN,NaN,INFO,User Authenticator #5
6578,2023-03-09,18:50:50,Server thread/INFO,ARTEXX joined the game,NaN,NaN,INFO,Server thread
6579,2023-03-09,18:50:50,Server thread/INFO,ARTEXX[/[0:0:0:0:0:0:0:1,NaN,NaN,INFO,Server thread
6580,2023-03-09,18:52:31,Server thread/INFO,ARTEXX issued server command: /farwand,NaN,NaN,INFO,Server thread


В таблице ниже видно, что мы корректно уалили теги сообщений.

In [ ]:
data_logs['serv_teg_INF'].value_counts()

Server thread              5444
Worker-Main-3               157
Worker-Main-4               156
Worker-Main-1               131
Worker-Main-6               120
Worker-Main-5               116
Worker-Main-7               107
Worker-Main-2               104
ServerMain                   86
User Authenticator #1        22
User Authenticator #2        13
WorldGuard Region I/O        11
User Authenticator #3        10
User Authenticator #4         9
User Authenticator #5         9
User Authenticator #6         8
User Authenticator #7         8
User Authenticator #8         5
Async Chat Thread - #0        5
Async Chat Thread - #2        5
Async Chat Thread - #6        3
Async Chat Thread - #10       3
User Authenticator #9         3
User Authenticator #10        2
Async Chat Thread - #8        1
Async Chat Thread - #4        1
User Authenticator #11        1
User Authenticator #12        1
User Authenticator #13        1
User Authenticator #14        1
Async Chat Thread - #1        1
Name: se

Проверим как они перенеслись в отдельный столбец.

In [ ]:
data_logs[['date', 'time', 'serv_teg', 'serv_teg_TEG', 'serv_teg_INF']]

,date,time,serv_teg,serv_teg_TEG,serv_teg_INF
0,2023-02-13,21:31:47,ServerMain/INFO,INFO,ServerMain
1,2023-02-13,21:31:48,ServerMain/ERROR,ERROR,ServerMain
2,2023-02-13,21:31:48,ServerMain/WARN,WARN,ServerMain
3,2023-02-13,21:31:48,ServerMain/INFO,INFO,ServerMain
5,2023-02-13,21:34:03,ServerMain/INFO,INFO,ServerMain
...,...,...,...,...,...
6577,2023-03-09,18:50:50,User Authenticator #5/INFO,INFO,User Authenticator #5
6578,2023-03-09,18:50:50,Server thread/INFO,INFO,Server thread
6579,2023-03-09,18:50:50,Server thread/INFO,INFO,Server thread
6580,2023-03-09,18:52:31,Server thread/INFO,INFO,Server thread


## 2.2 Категории сообщений

In [ ]:
# vibo: выделим информацию Worker-Main
data_logs['worker_main_number'] = data_logs['serv_teg_INF'].map(lambda x: 'Worker-Main' in x)


ind_WM = data_logs[data_logs['worker_main_number'] == True].index
data_logs['worker_main_number'] = data_logs[data_logs.index.isin(ind_WM)]['serv_teg_INF'].map(lambda x: x[12:])
data_logs['__serv_teg_INF1__'] = data_logs[data_logs.index.isin(ind_WM)]['serv_teg_INF'].map(lambda x: 'Worker-Main')

data_logs

,date,time,serv_teg,serv_mess,plug_name,plug_mess,serv_teg_TEG,serv_teg_INF,worker_main_number,__serv_teg_INF1__
0,2023-02-13,21:31:47,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO,ServerMain,NaN,NaN
1,2023-02-13,21:31:48,ServerMain/ERROR,Failed to load properties from file: server.p...,NaN,NaN,ERROR,ServerMain,NaN,NaN
2,2023-02-13,21:31:48,ServerMain/WARN,Failed to load eula.txt,NaN,NaN,WARN,ServerMain,NaN,NaN
3,2023-02-13,21:31:48,ServerMain/INFO,You need to agree to the EULA in order to run...,NaN,NaN,INFO,ServerMain,NaN,NaN
5,2023-02-13,21:34:03,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO,ServerMain,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
6577,2023-03-09,18:50:50,User Authenticator #5/INFO,UUID of player ARTEXX is ee86a614-d39f-3a50-a...,NaN,NaN,INFO,User Authenticator #5,NaN,NaN
6578,2023-03-09,18:50:50,Server thread/INFO,ARTEXX joined the game,NaN,NaN,INFO,Server thread,NaN,NaN
6579,2023-03-09,18:50:50,Server thread/INFO,ARTEXX[/[0:0:0:0:0:0:0:1,NaN,NaN,INFO,Server thread,NaN,NaN
6580,2023-03-09,18:52:31,Server thread/INFO,ARTEXX issued server command: /farwand,NaN,NaN,INFO,Server thread,NaN,NaN


In [ ]:
# vibo: сделаем также для User Authenticator
data_logs['user_authenticator_number'] = data_logs['serv_teg_INF'].dropna().map(lambda x: 'User Authenticator' in x)


ind_UA = data_logs[data_logs['user_authenticator_number'] == True].index
data_logs['user_authenticator_number'] = data_logs[data_logs.index.isin(ind_UA)]['serv_teg_INF'].map(lambda x: x[20:])
data_logs['__serv_teg_INF2__'] = data_logs[data_logs.index.isin(ind_UA)]['serv_teg_INF'].map(lambda x: 'User Authenticator')

data_logs

,date,time,serv_teg,serv_mess,plug_name,plug_mess,serv_teg_TEG,serv_teg_INF,worker_main_number,__serv_teg_INF1__,user_authenticator_number,__serv_teg_INF2__
0,2023-02-13,21:31:47,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO,ServerMain,NaN,NaN,NaN,NaN
1,2023-02-13,21:31:48,ServerMain/ERROR,Failed to load properties from file: server.p...,NaN,NaN,ERROR,ServerMain,NaN,NaN,NaN,NaN
2,2023-02-13,21:31:48,ServerMain/WARN,Failed to load eula.txt,NaN,NaN,WARN,ServerMain,NaN,NaN,NaN,NaN
3,2023-02-13,21:31:48,ServerMain/INFO,You need to agree to the EULA in order to run...,NaN,NaN,INFO,ServerMain,NaN,NaN,NaN,NaN
5,2023-02-13,21:34:03,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO,ServerMain,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6577,2023-03-09,18:50:50,User Authenticator #5/INFO,UUID of player ARTEXX is ee86a614-d39f-3a50-a...,NaN,NaN,INFO,User Authenticator #5,NaN,NaN,5,User Authenticator
6578,2023-03-09,18:50:50,Server thread/INFO,ARTEXX joined the game,NaN,NaN,INFO,Server thread,NaN,NaN,NaN,NaN
6579,2023-03-09,18:50:50,Server thread/INFO,ARTEXX[/[0:0:0:0:0:0:0:1,NaN,NaN,INFO,Server thread,NaN,NaN,NaN,NaN
6580,2023-03-09,18:52:31,Server thread/INFO,ARTEXX issued server command: /farwand,NaN,NaN,INFO,Server thread,NaN,NaN,NaN,NaN


In [ ]:
# vibo: и Async Chat Thread
data_logs['async_chat_thread_number'] = data_logs['serv_teg_INF'].dropna().map(lambda x: 'Async Chat Thread' in x)


ind_ACTN = data_logs[data_logs['async_chat_thread_number'] == True].index
data_logs['async_chat_thread_number'] = data_logs[data_logs.index.isin(ind_ACTN)]['serv_teg_INF'].map(lambda x: x[21:])
data_logs['__serv_teg_INF3__'] = data_logs[data_logs.index.isin(ind_ACTN)]['serv_teg_INF'].map(lambda x: 'Async Chat Thread')

data_logs

,date,time,serv_teg,serv_mess,plug_name,plug_mess,serv_teg_TEG,serv_teg_INF,worker_main_number,__serv_teg_INF1__,user_authenticator_number,__serv_teg_INF2__,async_chat_thread_number,__serv_teg_INF3__
0,2023-02-13,21:31:47,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO,ServerMain,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-02-13,21:31:48,ServerMain/ERROR,Failed to load properties from file: server.p...,NaN,NaN,ERROR,ServerMain,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-02-13,21:31:48,ServerMain/WARN,Failed to load eula.txt,NaN,NaN,WARN,ServerMain,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-02-13,21:31:48,ServerMain/INFO,You need to agree to the EULA in order to run...,NaN,NaN,INFO,ServerMain,NaN,NaN,NaN,NaN,NaN,NaN
5,2023-02-13,21:34:03,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO,ServerMain,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6577,2023-03-09,18:50:50,User Authenticator #5/INFO,UUID of player ARTEXX is ee86a614-d39f-3a50-a...,NaN,NaN,INFO,User Authenticator #5,NaN,NaN,5,User Authenticator,NaN,NaN
6578,2023-03-09,18:50:50,Server thread/INFO,ARTEXX joined the game,NaN,NaN,INFO,Server thread,NaN,NaN,NaN,NaN,NaN,NaN
6579,2023-03-09,18:50:50,Server thread/INFO,ARTEXX[/[0:0:0:0:0:0:0:1,NaN,NaN,INFO,Server thread,NaN,NaN,NaN,NaN,NaN,NaN
6580,2023-03-09,18:52:31,Server thread/INFO,ARTEXX issued server command: /farwand,NaN,NaN,INFO,Server thread,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# vibo: собираем в все в один столбик ffill (все собралось в последний столбец)
data_logs[['__serv_teg_INF1__', '__serv_teg_INF2__', '__serv_teg_INF3__']] = data_logs[['__serv_teg_INF1__', '__serv_teg_INF2__', '__serv_teg_INF3__']].ffill(axis=1)
data_logs

,date,time,serv_teg,serv_mess,plug_name,plug_mess,serv_teg_TEG,serv_teg_INF,worker_main_number,__serv_teg_INF1__,user_authenticator_number,__serv_teg_INF2__,async_chat_thread_number,__serv_teg_INF3__
0,2023-02-13,21:31:47,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO,ServerMain,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-02-13,21:31:48,ServerMain/ERROR,Failed to load properties from file: server.p...,NaN,NaN,ERROR,ServerMain,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-02-13,21:31:48,ServerMain/WARN,Failed to load eula.txt,NaN,NaN,WARN,ServerMain,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-02-13,21:31:48,ServerMain/INFO,You need to agree to the EULA in order to run...,NaN,NaN,INFO,ServerMain,NaN,NaN,NaN,NaN,NaN,NaN
5,2023-02-13,21:34:03,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO,ServerMain,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6577,2023-03-09,18:50:50,User Authenticator #5/INFO,UUID of player ARTEXX is ee86a614-d39f-3a50-a...,NaN,NaN,INFO,User Authenticator #5,NaN,NaN,5,User Authenticator,NaN,User Authenticator
6578,2023-03-09,18:50:50,Server thread/INFO,ARTEXX joined the game,NaN,NaN,INFO,Server thread,NaN,NaN,NaN,NaN,NaN,NaN
6579,2023-03-09,18:50:50,Server thread/INFO,ARTEXX[/[0:0:0:0:0:0:0:1,NaN,NaN,INFO,Server thread,NaN,NaN,NaN,NaN,NaN,NaN
6580,2023-03-09,18:52:31,Server thread/INFO,ARTEXX issued server command: /farwand,NaN,NaN,INFO,Server thread,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# vibo: проверяем, получилось как надо
data_logs['__serv_teg_INF3__'].value_counts()

Worker-Main           891
User Authenticator     93
Async Chat Thread      19
Name: __serv_teg_INF3__, dtype: int64

In [ ]:
# vibo: удаляем промежуточные столбцы
data_logs = data_logs.drop(['__serv_teg_INF1__', '__serv_teg_INF2__'], axis=1)
data_logs

,date,time,serv_teg,serv_mess,plug_name,plug_mess,serv_teg_TEG,serv_teg_INF,worker_main_number,user_authenticator_number,async_chat_thread_number,__serv_teg_INF3__
0,2023-02-13,21:31:47,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO,ServerMain,NaN,NaN,NaN,NaN
1,2023-02-13,21:31:48,ServerMain/ERROR,Failed to load properties from file: server.p...,NaN,NaN,ERROR,ServerMain,NaN,NaN,NaN,NaN
2,2023-02-13,21:31:48,ServerMain/WARN,Failed to load eula.txt,NaN,NaN,WARN,ServerMain,NaN,NaN,NaN,NaN
3,2023-02-13,21:31:48,ServerMain/INFO,You need to agree to the EULA in order to run...,NaN,NaN,INFO,ServerMain,NaN,NaN,NaN,NaN
5,2023-02-13,21:34:03,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO,ServerMain,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6577,2023-03-09,18:50:50,User Authenticator #5/INFO,UUID of player ARTEXX is ee86a614-d39f-3a50-a...,NaN,NaN,INFO,User Authenticator #5,NaN,5,NaN,User Authenticator
6578,2023-03-09,18:50:50,Server thread/INFO,ARTEXX joined the game,NaN,NaN,INFO,Server thread,NaN,NaN,NaN,NaN
6579,2023-03-09,18:50:50,Server thread/INFO,ARTEXX[/[0:0:0:0:0:0:0:1,NaN,NaN,INFO,Server thread,NaN,NaN,NaN,NaN
6580,2023-03-09,18:52:31,Server thread/INFO,ARTEXX issued server command: /farwand,NaN,NaN,INFO,Server thread,NaN,NaN,NaN,NaN


In [ ]:
# vibo: индексы ячеек, которые нужно перенести
find_ind = list(set(data_logs['serv_teg_INF'].index) - (set(ind_WM) | set(ind_UA) | set(ind_ACTN)))
# vibo: делаем новый столбец
data_logs['__serv_teg_INF4__'] = data_logs[data_logs.index.isin(find_ind)]['serv_teg_INF'].map(lambda x: x)

In [ ]:
data_logs

,date,time,serv_teg,serv_mess,plug_name,plug_mess,serv_teg_TEG,serv_teg_INF,worker_main_number,user_authenticator_number,async_chat_thread_number,__serv_teg_INF3__,__serv_teg_INF4__
0,2023-02-13,21:31:47,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO,ServerMain,NaN,NaN,NaN,NaN,ServerMain
1,2023-02-13,21:31:48,ServerMain/ERROR,Failed to load properties from file: server.p...,NaN,NaN,ERROR,ServerMain,NaN,NaN,NaN,NaN,ServerMain
2,2023-02-13,21:31:48,ServerMain/WARN,Failed to load eula.txt,NaN,NaN,WARN,ServerMain,NaN,NaN,NaN,NaN,ServerMain
3,2023-02-13,21:31:48,ServerMain/INFO,You need to agree to the EULA in order to run...,NaN,NaN,INFO,ServerMain,NaN,NaN,NaN,NaN,ServerMain
5,2023-02-13,21:34:03,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO,ServerMain,NaN,NaN,NaN,NaN,ServerMain
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6577,2023-03-09,18:50:50,User Authenticator #5/INFO,UUID of player ARTEXX is ee86a614-d39f-3a50-a...,NaN,NaN,INFO,User Authenticator #5,NaN,5,NaN,User Authenticator,NaN
6578,2023-03-09,18:50:50,Server thread/INFO,ARTEXX joined the game,NaN,NaN,INFO,Server thread,NaN,NaN,NaN,NaN,Server thread
6579,2023-03-09,18:50:50,Server thread/INFO,ARTEXX[/[0:0:0:0:0:0:0:1,NaN,NaN,INFO,Server thread,NaN,NaN,NaN,NaN,Server thread
6580,2023-03-09,18:52:31,Server thread/INFO,ARTEXX issued server command: /farwand,NaN,NaN,INFO,Server thread,NaN,NaN,NaN,NaN,Server thread


In [ ]:
# vibo: проверяем, получилось как надо
data_logs['__serv_teg_INF4__'].value_counts()

Server thread            5444
ServerMain                 86
WorldGuard Region I/O      11
Name: __serv_teg_INF4__, dtype: int64

In [ ]:
# vibo: собираем в все в один столбик ffill (все собралось в последний столбец)
data_logs[['__serv_teg_INF3__', '__serv_teg_INF4__']] = data_logs[['__serv_teg_INF3__', '__serv_teg_INF4__']].ffill(axis=1)
data_logs

,date,time,serv_teg,serv_mess,plug_name,plug_mess,serv_teg_TEG,serv_teg_INF,worker_main_number,user_authenticator_number,async_chat_thread_number,__serv_teg_INF3__,__serv_teg_INF4__
0,2023-02-13,21:31:47,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO,ServerMain,NaN,NaN,NaN,NaN,ServerMain
1,2023-02-13,21:31:48,ServerMain/ERROR,Failed to load properties from file: server.p...,NaN,NaN,ERROR,ServerMain,NaN,NaN,NaN,NaN,ServerMain
2,2023-02-13,21:31:48,ServerMain/WARN,Failed to load eula.txt,NaN,NaN,WARN,ServerMain,NaN,NaN,NaN,NaN,ServerMain
3,2023-02-13,21:31:48,ServerMain/INFO,You need to agree to the EULA in order to run...,NaN,NaN,INFO,ServerMain,NaN,NaN,NaN,NaN,ServerMain
5,2023-02-13,21:34:03,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO,ServerMain,NaN,NaN,NaN,NaN,ServerMain
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6577,2023-03-09,18:50:50,User Authenticator #5/INFO,UUID of player ARTEXX is ee86a614-d39f-3a50-a...,NaN,NaN,INFO,User Authenticator #5,NaN,5,NaN,User Authenticator,User Authenticator
6578,2023-03-09,18:50:50,Server thread/INFO,ARTEXX joined the game,NaN,NaN,INFO,Server thread,NaN,NaN,NaN,NaN,Server thread
6579,2023-03-09,18:50:50,Server thread/INFO,ARTEXX[/[0:0:0:0:0:0:0:1,NaN,NaN,INFO,Server thread,NaN,NaN,NaN,NaN,Server thread
6580,2023-03-09,18:52:31,Server thread/INFO,ARTEXX issued server command: /farwand,NaN,NaN,INFO,Server thread,NaN,NaN,NaN,NaN,Server thread


In [ ]:
# vibo: проверяем, получилось как надо
data_logs['__serv_teg_INF4__'].value_counts()

Server thread            5444
Worker-Main               891
User Authenticator         93
ServerMain                 86
Async Chat Thread          19
WorldGuard Region I/O      11
Name: __serv_teg_INF4__, dtype: int64

In [ ]:
# vibo: проверяем, что получилось, ок
data_logs

,date,time,serv_teg,serv_mess,plug_name,plug_mess,serv_teg_TEG,serv_teg_INF,worker_main_number,user_authenticator_number,async_chat_thread_number,__serv_teg_INF3__,__serv_teg_INF4__
0,2023-02-13,21:31:47,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO,ServerMain,NaN,NaN,NaN,NaN,ServerMain
1,2023-02-13,21:31:48,ServerMain/ERROR,Failed to load properties from file: server.p...,NaN,NaN,ERROR,ServerMain,NaN,NaN,NaN,NaN,ServerMain
2,2023-02-13,21:31:48,ServerMain/WARN,Failed to load eula.txt,NaN,NaN,WARN,ServerMain,NaN,NaN,NaN,NaN,ServerMain
3,2023-02-13,21:31:48,ServerMain/INFO,You need to agree to the EULA in order to run...,NaN,NaN,INFO,ServerMain,NaN,NaN,NaN,NaN,ServerMain
5,2023-02-13,21:34:03,ServerMain/INFO,Building unoptimized datafixer,NaN,NaN,INFO,ServerMain,NaN,NaN,NaN,NaN,ServerMain
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6577,2023-03-09,18:50:50,User Authenticator #5/INFO,UUID of player ARTEXX is ee86a614-d39f-3a50-a...,NaN,NaN,INFO,User Authenticator #5,NaN,5,NaN,User Authenticator,User Authenticator
6578,2023-03-09,18:50:50,Server thread/INFO,ARTEXX joined the game,NaN,NaN,INFO,Server thread,NaN,NaN,NaN,NaN,Server thread
6579,2023-03-09,18:50:50,Server thread/INFO,ARTEXX[/[0:0:0:0:0:0:0:1,NaN,NaN,INFO,Server thread,NaN,NaN,NaN,NaN,Server thread
6580,2023-03-09,18:52:31,Server thread/INFO,ARTEXX issued server command: /farwand,NaN,NaN,INFO,Server thread,NaN,NaN,NaN,NaN,Server thread


In [ ]:
# vibo: удаляем лишнее
data_logs = data_logs.drop(['serv_teg', 'serv_teg_INF', '__serv_teg_INF3__'], axis=1)
# vibo: переименовываем
data_logs = data_logs.rename(columns={"__serv_teg_INF4__": "serv_teg_inf"})
data_logs = data_logs.rename(columns={"serv_teg_TEG": "serv_teg"})

In [ ]:
# vibo: выстраиваем в нужном нам порядке
data_logs = data_logs[['date', 'time', 'serv_teg_inf', 'serv_teg', 'serv_mess', 'user_authenticator_number', 'worker_main_number', 'async_chat_thread_number', 'plug_name', 'plug_mess']]
data_logs

,date,time,serv_teg_inf,serv_teg,serv_mess,user_authenticator_number,worker_main_number,async_chat_thread_number,plug_name,plug_mess
0,2023-02-13,21:31:47,ServerMain,INFO,Building unoptimized datafixer,NaN,NaN,NaN,NaN,NaN
1,2023-02-13,21:31:48,ServerMain,ERROR,Failed to load properties from file: server.p...,NaN,NaN,NaN,NaN,NaN
2,2023-02-13,21:31:48,ServerMain,WARN,Failed to load eula.txt,NaN,NaN,NaN,NaN,NaN
3,2023-02-13,21:31:48,ServerMain,INFO,You need to agree to the EULA in order to run...,NaN,NaN,NaN,NaN,NaN
5,2023-02-13,21:34:03,ServerMain,INFO,Building unoptimized datafixer,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
6577,2023-03-09,18:50:50,User Authenticator,INFO,UUID of player ARTEXX is ee86a614-d39f-3a50-a...,5,NaN,NaN,NaN,NaN
6578,2023-03-09,18:50:50,Server thread,INFO,ARTEXX joined the game,NaN,NaN,NaN,NaN,NaN
6579,2023-03-09,18:50:50,Server thread,INFO,ARTEXX[/[0:0:0:0:0:0:0:1,NaN,NaN,NaN,NaN,NaN
6580,2023-03-09,18:52:31,Server thread,INFO,ARTEXX issued server command: /farwand,NaN,NaN,NaN,NaN,NaN


Закончили разбор заголовка сообщений сервера. Сохраняемся.

In [ ]:
# vibo: сохраняем предобработанный лог (по дефолту index = True, сохраняется)
data_logs.to_csv('/content/drive/MyDrive/minecraft/logs.csv')